Derogatory Comment Detection In Bengali Language Using Transformer Models



In [ ]:
! pip install tensorboardX pandas simpletransformers transformers

In [ ]:
!pip install transformers

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

In [ ]:
pip install bnemo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from bnemo import Translator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/study /NLP 431/Hate-Speech-Master-File (1).csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30853 entries, 0 to 30852
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sl No       30853 non-null  int64  
 1   comment     30853 non-null  object 
 2   label       30853 non-null  object 
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   Unnamed: 5  0 non-null      float64
 6   Unnamed: 6  0 non-null      float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.6+ MB


In [ ]:
df.head()

,Sl No,comment,label,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1,মাগি তরে কচু দেওয়ার দরকার,bully,NaN,NaN,NaN,NaN
1,2,আলম ভাই এই করোনা ভাইরাস এর সময় ছবি রিলিজ দিওন...,not bully,NaN,NaN,NaN,NaN
2,3,"আরেএ ভাই?৪ফিট দুরত্ব রাখেন,নাহয় তো করোনাই আপনা...",not bully,NaN,NaN,NaN,NaN
3,4,কুকুর কে দু* খাওয়াচ্ছো নাকি,bully,NaN,NaN,NaN,NaN
4,5,মাগি চুদাচুদি করার আর মানুষ পেলনা,bully,NaN,NaN,NaN,NaN


In [ ]:
df.rename(columns = {'comment':'text'}, inplace = True)

In [ ]:
df.rename(columns = {'label':'labels'}, inplace = True)

In [ ]:
unnamed_cols  =  df.columns.str.contains('Unnamed')
unnamed_cols

array([False, False, False,  True,  True,  True,  True])

In [ ]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
df = df.drop(columns=['Sl No'])
df.head()

,text,labels
0,মাগি তরে কচু দেওয়ার দরকার,bully
1,আলম ভাই এই করোনা ভাইরাস এর সময় ছবি রিলিজ দিওন...,not bully
2,"আরেএ ভাই?৪ফিট দুরত্ব রাখেন,নাহয় তো করোনাই আপনা...",not bully
3,কুকুর কে দু* খাওয়াচ্ছো নাকি,bully
4,মাগি চুদাচুদি করার আর মানুষ পেলনা,bully


In [ ]:
df['labels'].replace(['bully', 'not bully'], [1, 0], inplace=True)
df.head()

,text,labels
0,মাগি তরে কচু দেওয়ার দরকার,1
1,আলম ভাই এই করোনা ভাইরাস এর সময় ছবি রিলিজ দিওন...,0
2,"আরেএ ভাই?৪ফিট দুরত্ব রাখেন,নাহয় তো করোনাই আপনা...",0
3,কুকুর কে দু* খাওয়াচ্ছো নাকি,1
4,মাগি চুদাচুদি করার আর মানুষ পেলনা,1


In [ ]:
nltk.download('stopwords') #######
nltk.download('punkt')
stop_word_lt = (stopwords.words('bengali')) ########

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
stop_word_list = (stopwords.words('bengali'))

In [ ]:
print(stop_word_list)

['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও'

In [ ]:
print(len(stop_word_list))

398


In [ ]:
df.head(10)

,text,labels
0,মাগি তরে কচু দেওয়ার দরকার,1
1,আলম ভাই এই করোনা ভাইরাস এর সময় ছবি রিলিজ দিওন...,0
2,"আরেএ ভাই?৪ফিট দুরত্ব রাখেন,নাহয় তো করোনাই আপনা...",0
3,কুকুর কে দু* খাওয়াচ্ছো নাকি,1
4,মাগি চুদাচুদি করার আর মানুষ পেলনা,1
5,সিংগাপুর থেকে দেখতেছি,0
6,নাইচ বেব্বি।,0
7,ধন্যবাদ ক্ষমা চাউআর জন্য কিন্তু কার কাছে .....,0
8,আমি ফেইসবুক কমেন্ট কারিদের কাছে জানতে চাই ওনি ...,1
9,মেকাপ সুন্দরী,1


In [ ]:
dfE = dff.copy()
df1 = dff.copy()
df2 = dff

In [ ]:
def text_to_word_list(text):
    text = text.split()
    return text

def emoji(text):
  Etranslator = Translator()
  result = Etranslator.translate(text)

  return result.text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(t1):
  whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
  bangla_fullstop = u"\u0964"
  punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
  punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,৳০১২৩৪৫৬৭৮৯012–34567•89।'“\’,<>./?@#$%^&*_~‘—॥”‰¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"
  t1 = whitespace.sub(" ",t1).strip()
  t1 = re.sub(punctSeq, " ", t1)
  t1 = re.sub(bangla_fullstop, " ",t1)
  t1 = re.sub(punc, " ", t1)
  return t1



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out = remove_punctuations(replace_strings(text))
    return out

In [ ]:
df['text'] = df.text.apply(lambda x: preprocessing(str(x)))

In [ ]:
# dff.head(20)

In [ ]:
def remove_punctuations(t1):
  whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
  bangla_fullstop = u"\u0964"
  punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
  punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,৳০১২৩৪৫৬৭৮৯012–34567•89।'“\’,<>./?@#$%^&*_~‘—॥”‰¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"
  t1 = whitespace.sub(" ",t1).strip()
  t1 = re.sub(punctSeq, " ", t1)
  t1 = re.sub(bangla_fullstop, " ",t1)
  t1 = re.sub(punc, " ", t1)
  return t1

In [ ]:
df['text'] = df.text.apply(lambda x: remove_punctuations(str(x)))

In [ ]:
df.head(20)

,text,labels
0,মাগি তরে কচু দেওয়ার দরকার,1
1,আলম ভাই এই করোনা ভাইরাস এর সময় ছবি রিলিজ দিওন...,0
2,আরেএ ভাই ফিট দুরত্ব রাখেন নাহয় তো করোনাই আপনার...,0
3,কুকুর কে দু খাওয়াচ্ছো নাকি,1
4,মাগি চুদাচুদি করার আর মানুষ পেলনা,1
5,সিংগাপুর থেকে দেখতেছি,0
6,নাইচ বেব্বি,0
7,ধন্যবাদ ক্ষমা চাউআর জন্য কিন্তু কার কাছে,0
8,আমি ফেইসবুক কমেন্ট কারিদের কাছে জানতে চাই ওনি ...,1
9,মেকাপ সুন্দরী,1


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
k = 4

In [ ]:
kf = KFold(n_splits=k)

In [ ]:
for train_index, test_index in kf.split(df):
    train_data = df.iloc[train_index]
    test_data = df.iloc[test_index]

In [ ]:
train_data

,text,labels
0,মাগি তরে কচু দেওয়ার দরকার,1
1,আলম ভাই এই করোনা ভাইরাস এর সময় ছবি রিলিজ দিওন...,0
2,আরেএ ভাই ফিট দুরত্ব রাখেন নাহয় তো করোনাই আপনার...,0
3,কুকুর কে দু খাওয়াচ্ছো নাকি,1
4,মাগি চুদাচুদি করার আর মানুষ পেলনা,1
...,...,...
23135,এই এশিয়া মহাদেশে তোমার উপরে কেও নাই,0
23136,এই রেশমি মাগী তুই কি এখনও মরস নাইতুই মরলে আমি ...,1
23137,আপনি যদি বছর পর পোস্ট করেন তবুও কি হা হা রিয়াক...,0
23138,হিরো আলম কে ধন্যবাদ আমরা তোমার সাথে আছি থাকবো,0


In [ ]:
len(test_data)

7713

In [ ]:
bert2 = ClassificationModel('bert', 'sagorsarker/bangla-bert-base', num_labels=2, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,

})
bert2.train_model(train_data)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/23140 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2893 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2893 [00:00<?, ?it/s]

(5786, 0.2532397486163883)

In [ ]:
result, model_outputs, wrong_predictions = bert2.eval_model(test_data)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions:")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test_data)))
bads

  0%|          | 0/7713 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/965 [00:00<?, ?it/s]

wrong predictions:
514 wrong out of 7713


{0: 247, 1: 267}

In [ ]:
print("Accuracy %")
(len(test_data)-len(wrong_predictions))/len(test_data)*100

Accuracy %


93.33592635809673

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
for i in range(2):

    # Evaluate the model
    result, model_outputs, wrong_predictions = bert2.eval_model(test_data)
    print("Accuracy= " ,(result['tp'] + result['tn']) / (result['tp'] + result['tn'] + \
                                                         result['fp'] + result['fn']))
    print("Recall = ",(result['tn']) / (result['tn'] + result['fn'])) # simpletransformers mistakenly reports fn and fp. You need top flip them
    print(result)
    print(classification_report(np.argmax(model_outputs, axis = 1), test_data.labels.values))

  0%|          | 0/7713 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/965 [00:00<?, ?it/s]

Accuracy=  0.9333592635809672
Recall =  0.9349573690621193
{'mcc': 0.8662194580677559, 'tp': 3361, 'tn': 3838, 'fp': 247, 'fn': 267, 'auroc': 0.9806811633709797, 'auprc': 0.9801152633557496, 'eval_loss': 0.2991723368822602}
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      4105
           1       0.93      0.93      0.93      3608

    accuracy                           0.93      7713
   macro avg       0.93      0.93      0.93      7713
weighted avg       0.93      0.93      0.93      7713



  0%|          | 0/7713 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/965 [00:00<?, ?it/s]

Accuracy=  0.9333592635809672
Recall =  0.9349573690621193
{'mcc': 0.8662194580677559, 'tp': 3361, 'tn': 3838, 'fp': 247, 'fn': 267, 'auroc': 0.9806811633709797, 'auprc': 0.9801152633557496, 'eval_loss': 0.2991723368822602}
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      4105
           1       0.93      0.93      0.93      3608

    accuracy                           0.93      7713
   macro avg       0.93      0.93      0.93      7713
weighted avg       0.93      0.93      0.93      7713

